# TRIQS Green's functions

It is now time to start using some of the tools provided by TRIQS.

Much of the functionality in TRIQS, while implemented in C++ for optimal performance, is exposed
through a Python interface to make it easier to use. From a practical point of view this means
that you can think of TRIQS as a python library, just like numpy or matplotlib.

One of the central objects of a many-body calculation is a Green's function.
Green's functions in TRIQS are functions defined on a mesh $\cal{M}$ of points that hold values in some domain $\cal{D}$, for example $\mathbb{C}^{2\times2}$

$$
G: \cal{M} \rightarrow \cal{D}
$$

A few common Green's function meshes in TRIQS include:

- `MeshReFreq` - Real-frequencies equally spaced in $[\omega_{min},\omega_{max}]$
- `MeshImFreq` - Matsubara Frequencies
- `MeshImTime` - Imaginary time points equally spaced in $[0,\beta]$
- `MeshReTime` - Real-time points (not covered in this tutorial)

Let's see how we can **construct a Mesh and print its values**.

In [ ]:
# Import the Mesh type we want to use
from triqs.gf import MeshImTime

# The documentation tells us which parameters we need to pass for the mesh construction
?MeshImTime

In [ ]:
# Provide the inverse temperature, Statistic, and number of points
tau_mesh = MeshImTime(beta=5, statistic='Fermion', n_tau=11)

# We can loop and print the mesh-point values
for tau in tau_mesh:
    print(tau.value)

    # Using tab for auto completion can be very helpful to understand
    # which other members, functions and properties are available for
    # a given Python object like 'tau'.
    # Type 'tau.' below and use tab to see which options you get!

Let us now **create and initialize a Green's function for a single atomic level** with energy $\epsilon$ in the grand-canonical ensemble with inverse temperature $\beta$

$$
G[\tau] = -\langle\cal{T}c(\tau) c^\dagger\rangle = -\frac{e^{-\tau \epsilon}}{1+e^{-\beta \epsilon}}
= -\frac{e^{\left(\beta \Theta(-\epsilon) -\tau\right) \epsilon}}{1+e^{-\beta |\epsilon|}}$$

In practice we use the second expression, as it avoids diverging exponentials for large values of $\beta$.
We first have a look at the documentation for `Gf`.

In [ ]:
from triqs.gf import Gf
?Gf

In [ ]:
# Create scalar-valued imaginary-time Green's function
G = Gf(mesh=tau_mesh, target_shape=[], is_real=True)

# Print the Green's function description
print(G)

In [ ]:
# Loop initialization
eps = -0.4
beta = G.mesh.beta
from math import exp
for tau in G.mesh:
    G[tau] = -exp((beta*(eps<0) - tau.value) * eps) / (1. + exp(-beta * abs(eps)))
    print(f"{G[tau]:.3f}")

In order to **plot this Green's function** we can use the matplotlib interface defined in TRIQS.
Note that the function to plot Green's function is `oplot` and not just `plot` like in matplotlib.

In [ ]:
from triqs.plot.mpl_interface import oplot,plt

# Make plots show up directly in the notebook:
%matplotlib inline

# Make all figures slightly bigger
import matplotlib as mpl
mpl.rcParams['figure.dpi']=100

# Additional arguments like 'linewidth' are passed on to matplotlib
oplot(G, '-', name='G', linewidth=2)

## Matrix-Valued Green's functions

In most realistic problems we have to treat more than just a single orbital

$$
G_{ij}[\tau] = -\langle\cal{T}c_i(\tau) c_j^\dagger\rangle
$$

For this purpose, TRIQS provides Green's functions that have a Matrix structure. Let's see how you can create and use them

In [ ]:
# A uniform real-frequency mesh on a given interval
from triqs.gf import MeshReFreq
w_mesh = MeshReFreq(window=(-4,4), n_w=1000)

# Gf with 2x2 Matrix structure holding complex values
G = Gf(mesh=w_mesh, target_shape=[2,2])
G # <- Same as print(repr(G))

In [ ]:
# Accessing a specific mesh point gives us a matrix
from triqs.gf import Idx # Use Idx to access Gf at specific Index
print(G[Idx(0)])

In [ ]:
# By Fixing the orbital indices we obtain a Green's function that is no longer matrix but complex-valued
G[0,0]

## Additional Initialization Descriptors

In the following we will introduce a few additional means of initializing Green's functions using `<<`.

### Flat density of states
Consider the problem of a single atomic level embedded in a flat conduction bath $\Gamma$ of electrons.
$$
G(\omega) = \frac{1}{\omega - \epsilon_d - V^2 \Gamma(\omega) + i\eta}
$$

In the equation above $\epsilon_d$ is the energy of the level and $\Gamma$ is the Green's function of
a flat conduction bath

$$
\Gamma(\omega) = \int_{-D}^{D}\frac{1}{\omega-\epsilon + i\eta}\frac{d\epsilon}{2D}
$$

Here $D$ denotes the half-bandwidth.
Let's see how to define and then plot this Green's function by using `inverse` and the `Flat` descriptor.

First, we plot the flat density of states.

In [ ]:
D = 1.5 # Half bandwidth

G = Gf(mesh=w_mesh, target_shape=[])

from triqs.gf import Flat
from math import pi
G << Flat(D)

oplot(-G.imag/pi, name=r"$\rho$") 

Note the predefined function `Flat` for a flat conduction bath $\Gamma(\omega)$. Now, we calculate the actual Green function of the atomic level, hybridized with the flat bath.

In [ ]:
eps_d = 0.5   # Energy
V = 0.2       # Bath Hybridization
D = 1.5       # Half bandwidth
ieta = 1e-13j # small broadening

G = Gf(mesh=w_mesh, target_shape=[])

from triqs.gf import Omega, Flat, inverse
G << inverse(Omega - eps_d - V**2 * Flat(D) + ieta)

Let's plot the atomic Green's function. Note that default, both the real and imaginary parts are plotted.

In [ ]:
oplot(G, '-', linewidth=2, name="G") 

We can plot the spectral function, which is defined as

$$ \rho(\omega) = -\frac{1}{\pi} \, \textbf{Im} \, G $$

In [ ]:
oplot(-G.imag/pi, linewidth=2, name=r"$\rho$")

As expected the spectral function is peaked at $\epsilon_d$ and shows a jump in spectral weight at $D$.

### Semi-circular density of states

Another predefined Green's function is the one corresponding to a semi-circular spectral function. This one will be useful in the DMFT Tutorials later on.

In [ ]:
D = 1.0 # Half bandwidth

G = Gf(mesh=w_mesh, target_shape=[])

from triqs.gf import SemiCircular
G << SemiCircular(D)

oplot(-G.imag/pi, name=r"$\rho$") 

Imaginary-frequency Green's functions
-------------------------------------

These are Green's function defined on the Matsubara axis. The fermionic Matsubara frequencies
are defined by

$$\omega_n = \frac{(2n+1)\pi}{\beta}$$

where $\beta = 1/T$ is the inverse temperature. These Green's functions are important because
most Monte Carlo algorithms yield results on the Matsubara axis. Let's see how they
are defined:

In [ ]:
# Define the imaginary-frequency mesh
from triqs.gf import MeshImFreq
iw_mesh = MeshImFreq(beta=5, statistic='Fermion', n_iw=1000)

# Create Green's function and fill it using the iOmega_n descriptor
G = Gf(mesh=iw_mesh, target_shape=[])
from triqs.gf import iOmega_n
G << inverse(iOmega_n - 0.2)

# Plot the Green's function
oplot(G, '-o', name='G')
plt.xlim(0,10)
plt.show()

## Arithmetic Operations

Green's functions can be added, multiplied by numbers, etc. The way this is done is quite natural.

In [ ]:
oplot(G, "-o", name='G')
oplot(G+G, "-o", name='G+G')
oplot(3*G+2, "-o", name='3*G+2')
plt.xlim(0,10)
plt.show()

## Obtaining the density

You can obtain the density for Green's functions with a `MeshReFreq` and `MeshImFreq` using the `density` method

In [ ]:
G = Gf(mesh=iw_mesh, target_shape=[])
G << inverse(iOmega_n - 0.2)
print("Density =", G.density())

Do not worry about the imaginary component as the machine precision is on the order of $10^{-15}$.

## Fourier transforms

TRIQS allows you to easily Fourier transform Green's functions from imaginary-time to imaginary-frequency.

In [ ]:
# A Green's function in frequency set to semi-circular
Giw = Gf(mesh=iw_mesh, target_shape=[])
Giw << SemiCircular(1.0)

# A Green's function in time set by inverse Fourier transform
from triqs.gf import make_gf_from_fourier
Gtau = make_gf_from_fourier(Giw)
oplot(Gtau, name='G')

We can also go the other way. Let's check that it gives back the original result.

In [ ]:
Giw_2 = make_gf_from_fourier(Gtau)
oplot(Giw, 'o')
oplot(Giw_2, 'x')
plt.ylabel(r'$G(i\omega_n)$')
plt.xlim(0,5)
plt.show()

In the example above `make_gf_from_fourier` will construct a new Green's function object.
This uses the `statistic` information of `MeshImTime` to decide wether to use bosonic or fermionic Matsubara frequencies.
If we want to instead use an existing Green's function we can use the `Fourier` descriptor

In [ ]:
from triqs.gf import Fourier
Gtau << Fourier(Giw)

## Pade analytical continuation

The Fourier transforms allow to go from time to frequency. A much more delicate thing is to do the so-called "analytical continuation". This means to start from a Matsubara-frequency Green's function and obtain the corresponding real-frequency Green's function. This can formally be done, but turns out to be a mathematically ill-conditioned problem. Even small amounts of noise in the Matsubara-frequency data will make the continuation to the real axis very unstable.

One of the ways to do perform analytical continuation is to use [Pade approximants](https://en.wikipedia.org/wiki/Padé_approximant#Definition). TRIQS can do that for you in the following way:

*Note:* Pade is currently implemented only for Green's functions with a Matrix structure

In [ ]:
# The Matsubara Green's function to be continued
iw_mesh = MeshImFreq(beta=50, statistic='Fermion', n_iw=1000)
Giw = Gf(mesh=iw_mesh, target_shape=[1,1])
Giw << SemiCircular(1.0)

# Construct real-frequency Green's function and initialize it using Pade approximants
Gw = Gf(mesh=w_mesh, target_shape=[1,1])
Gw.set_from_pade(Giw)

oplot(-Gw.imag/pi, linewidth=2)

The coarse Matsubara discretization at high temperatures will worsen the Pade result, which is why we chose a much lower temperature value for this example.

You can see that the Pade continuation did a pretty good job. We will see later that noise will completely change this picture!

## Exercises

### <i class="fa fa-gear fa-x" style="color: #186391"></i> Exercise 1

Plot the density $n(\epsilon)$ as a function of $\epsilon$ for a Green's function $G = 1/(i\omega_n - \epsilon)$. What is the curve that you obtained? How does it change with temperature?

### <i class="fa fa-gear fa-x" style="color: #186391"></i> Exercise 2

Consider a Hubbard atom with $U=2$ at temperature $T = 1/\beta = 1/10$. The non-interacting and interacting Green's functions for this problem are:

$$
G_0 = \frac{1}{i \omega_n + \mu} \qquad \mu = U/2
$$

$$
G = \frac{1}{2(i\omega_n + U/2)} + \frac{1}{2(i\omega_n - U/2)}
$$

Here the chemical potential $\mu$ is chosen such that the interacting system is half filled.

Using Dyson's equation, verify that the corresponding self-energy is indeed

$$
\Sigma = \frac{U}{2} + \frac{U^2}{4 i\omega_n}
$$

*Note: At half-filling the chemical potential $\mu = U/2$ and the static part of the self-energy exactly cancel.*